<a href="https://colab.research.google.com/github/sdoshi12/TRP_site_code/blob/main/TRP_All_Groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title TRP 2 Groups

import pandas as pd #pandas python library takes data from excel & puts into rows for code to read
import numpy as np
import io #helps with file reading/writing
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
from random import seed
from random import random

input('Hit ENTER to run program')

print("Welcome to the Tumor Randomization Program")
print('\n')
#print("Please enter the file name below like this example --> wlabdata.xlsx")

from google.colab import files
uploaded = files.upload()

file_path = 'randomization_file.xlsx'
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
file_path

#pd.set_option('max_rows', 10)
#pd.set_option('display.max_columns', None)

df = pd.read_excel(io.BytesIO(uploaded[str(file_path)]))
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
print(df)
#------------------volume column---------------------------------
print('\n')
#vol_col = df["volume"]
volumes = df['volume']
volumes
#------------------ID column---------------------------------
id_col = df['Combined ID']
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
array = []
for i in df['volume']:
    array.append(i)
#------------------sorting array and formatting---------------------------------
array.sort()
array
new_array = []
for i in range(len(array)):
    new_array.append(array[len(array)-i-1])
    new_array.append(array[i])
    i+=1
new_array
val1 = len(array)
val2 = len(new_array)
new_array = np.delete(new_array, [range(val1,val2)])

new_df = pd.DataFrame(new_array)
new_df
#------------------grouping---------------------------------
#grouping
#group_num = input("Please enter number of groups: ")
group_num = int(2)

#groups =[]
#group_vals = []
#for i in range(group_num):
#  group_vals.append(1)
#group_vals
#group_inter = []
#group_inter.append(group_vals*len(df['volume']))
#for i in range(len(df['volume'])):
#  groups.append(group_inter[0][i])

#groups

# q = len(new_df)/2
# import random

# groups = []
# if int(len(new_df))%2 == 0:
#   len(new_df)/2
#   for i in range(int(q)):
#     groups.append(1)
#   for i in range(int(q)):
#     groups.append(2)
# else: #int(len(new_df))%2 != 0:
#   for i in range(int(q)):
#     groups.append(1)
#   for i in range(int(q)):
#     groups.append(2)
#   groups.append(random.randint(1,2))
group_num = int(2)
groups = []
new_group = [1,2,2,1]
group_intermediate = []
group_intermediate.append(new_group*len(df['volume']))

groups = []
for i in range(len(df['volume'])):
  groups.append(group_intermediate[0][i])



new_df["Groups"] = groups
print(new_df)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#sorting
sorted_df = new_df.sort_values("Groups")

df.index = df['volume']
id_df = df.drop(['volume'], axis = 'columns')
id_dict = id_df.to_dict()
print('--------ID DICT--------')
print(id_dict)
id_array = []
for i in sorted_df[0]:
  id_array.append(id_dict['Combined ID'][i])

sorted_df['Mouse ID'] = id_array
print(sorted_df)

stat_df = sorted_df.groupby('Groups')
#------------------statistics---------------------------------
mean = stat_df.mean()
mean = mean.to_numpy()
mean = mean.tolist()
median = stat_df.median()
median = median.to_numpy()
median = median.tolist()
std_dev = stat_df.std()
std_dev = std_dev.to_numpy()
std_dev = std_dev.tolist()

for i in range(len(sorted_df.index)-len(mean)):
  mean.append(0)
  median.append(0)
  std_dev.append(0)

sorted_df['Mean'] = mean
sorted_df['Median'] = median
sorted_df['Std_Dev'] = std_dev

#------------------ANOVA---------------------------------
##Stuff for the anova
anova_df = sorted_df.groupby('Groups', 0)
med = []
for i in anova_df:
  med.append(i)

anova_stat = []
for i in range(group_num):
  thing = med[i][1][0].to_numpy()
  anova_stat.append(thing)

anova_stat

print('Follow instructions below for ANOVA testing')
print('Within the code below, change the f_oneway variable to calculate the an accurate statistic. Input should look like: f_oneway(anova_stat[0], anova_stat[1], etc...')
print('The number of inputs depends on how many groups you have.')
print('\n')
arr = np.zeros(len(groups))
###-----------------------------------------MAKE SURE THE LINE BELOW IS RIGHT FOR ANOVA TESTING---------------------------------------------
anova_fvalue, anova_pvalue = f_oneway(anova_stat[0],anova_stat[1])
print("ANOVA p-value: " + str(anova_pvalue))
arr[0] = anova_pvalue
#print(arr)
sorted_df['ANOVA'] = arr
#------------------Kruskal-Wallis---------------------------------
##COPY AND PASTE THE VALUES FROM THE F_ONEWAY LINE INSIDE THE KRUSKAL LINE PLEASE
arr1 = np.zeros(len(groups))
kru_fvalue, kru_pvalue = stats.kruskal(anova_stat[0],anova_stat[1])
print("Kruskal p-value: " + str(kru_pvalue))
arr1[0] = kru_pvalue
sorted_df['Kruskal Wallis'] = arr1

#------------------percentage difference---------------------
avg = sorted_df[0].mean()
avg

graph_array_mean = []
for i in range(group_num):
  graph_array_mean.append(mean[i][0])
graph_array_std = []
for i in range(group_num):
  graph_array_std.append(std_dev[i][0])

diff_array = []
for i in graph_array_mean:
  diff = avg - i
  val = abs(diff/avg)*100
  diff_array.append(val)

for i in range(len(sorted_df)-int(group_num)):
  diff_array.append(0)

sorted_df['Difference Percentages'] = diff_array
#------------------difference coefficient---------------------------------
#difference coefficient
diff_coefficient = []
for i in range(group_num):
  diff_coefficient.append((graph_array_std[i]/graph_array_mean[i])*100)

for i in range(len(sorted_df)-int(group_num)):
  diff_coefficient.append(0)
sorted_df['Difference Coefficient'] = diff_coefficient

#------------------visualization---------------------------------
## FOR VISUALIZATION GRAPH
x = []
for i in range(group_num):
  x.append("Group "+ str(i+1))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, graph_array_mean, color='blue', yerr=graph_array_std)
plt.xlabel("Group Number")
plt.ylabel("Tumor Volume")
plt.title("Average Tumor Volume")

plt.xticks(x_pos, x)

plt.show()
#------------------saving the file---------------------------------
print('\n')
filename = input('Enter output filename (ie. output.csv): ')
print('Filename: '+ filename)
sorted_df.to_csv(filename)
files.download(filename)



In [ ]:
#@title TRP 3 Groups { display-mode: "form" }
import pandas as pd
import numpy as np
import io
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

input('Hit ENTER to run program')

print("Welcome to the Tumor Randomization Program")
print('\n')
#print("Please enter the file name below like this example --> wlabdata.xlsx")

from google.colab import files
uploaded = files.upload()

file_path = 'randomization_file.xlsx'
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
file_path

#pd.set_option('max_rows', 10)
#pd.set_option('display.max_columns', None)

df = pd.read_excel(io.BytesIO(uploaded[str(file_path)]))
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
print(df)
#------------------volume column---------------------------------
print('\n')
#vol_col = df["volume"]
volumes = df['volume']
volumes
#------------------ID column---------------------------------
id_col = df['Combined ID']
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
array = []
for i in df['volume']:
    array.append(i)
#------------------sorting array and formatting---------------------------------
array.sort()
array
new_array = []
for i in range(len(array)):
    new_array.append(array[len(array)-i-1])
    new_array.append(array[i])
    i+=1
new_array
val1 = len(array)
val2 = len(new_array)
new_array = np.delete(new_array, [range(val1,val2)])

new_df = pd.DataFrame(new_array)
new_df
#------------------grouping---------------------------------
#grouping
#group_num = input("Please enter number of groups: ")
group_num = int(3)

groups =[]
group_vals = []
for i in range(group_num):
  group_vals.append(i+1)
group_vals
group_inter = []
group_inter.append(group_vals*len(df['volume']))
for i in range(len(df['volume'])):
  groups.append(group_inter[0][i])

new_df["Groups"] = groups
print(new_df)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#sorting
sorted_df = new_df.sort_values("Groups")

df.index = df['volume']
id_df = df.drop(['volume'], axis = 'columns')
id_dict = id_df.to_dict()
print('--------ID DICT--------')
print(id_dict)
id_array = []
for i in sorted_df[0]:
  id_array.append(id_dict['Combined ID'][i])

sorted_df['Mouse ID'] = id_array
print(sorted_df)

stat_df = sorted_df.groupby('Groups')
#------------------statistics---------------------------------
mean = stat_df.mean()
mean = mean.to_numpy()
mean = mean.tolist()
median = stat_df.median()
median = median.to_numpy()
median = median.tolist()
std_dev = stat_df.std()
std_dev = std_dev.to_numpy()
std_dev = std_dev.tolist()

for i in range(len(sorted_df.index)-len(mean)):
  mean.append(0)
  median.append(0)
  std_dev.append(0)

sorted_df['Mean'] = mean
sorted_df['Median'] = median
sorted_df['Std_Dev'] = std_dev

#------------------ANOVA---------------------------------
##Stuff for the anova
anova_df = sorted_df.groupby('Groups', 0)
med = []
for i in anova_df:
  med.append(i)

anova_stat = []
for i in range(group_num):
  thing = med[i][1][0].to_numpy()
  anova_stat.append(thing)

anova_stat

print('Follow instructions below for ANOVA testing')
print('Within the code below, change the f_oneway variable to calculate the an accurate statistic. Input should look like: f_oneway(anova_stat[0], anova_stat[1], etc...')
print('The number of inputs depends on how many groups you have.')
print('\n')
arr = np.zeros(len(groups))
###-----------------------------------------MAKE SURE THE LINE BELOW IS RIGHT FOR ANOVA TESTING---------------------------------------------
anova_fvalue, anova_pvalue = f_oneway(anova_stat[0],anova_stat[1], anova_stat[2])
print("ANOVA p-value: " + str(anova_pvalue))
arr[0] = anova_pvalue
#print(arr)
sorted_df['ANOVA'] = arr
#------------------Kruskal-Wallis---------------------------------
##COPY AND PASTE THE VALUES FROM THE F_ONEWAY LINE INSIDE THE KRUSKAL LINE PLEASE
arr1 = np.zeros(len(groups))
kru_fvalue, kru_pvalue = stats.kruskal(anova_stat[0],anova_stat[1], anova_stat[2])
print("Kruskal p-value: " + str(kru_pvalue))
arr1[0] = kru_pvalue
sorted_df['Kruskal Wallis'] = arr1

#------------------percentage difference---------------------
avg = sorted_df[0].mean()
avg

graph_array_mean = []
for i in range(group_num):
  graph_array_mean.append(mean[i][0])
graph_array_std = []
for i in range(group_num):
  graph_array_std.append(std_dev[i][0])

diff_array = []
for i in graph_array_mean:
  diff = avg - i
  val = abs(diff/avg)*100
  diff_array.append(val)

for i in range(len(sorted_df)-int(group_num)):
  diff_array.append(0)

sorted_df['Difference Percentages'] = diff_array
#------------------difference coefficient---------------------------------
#difference coefficient
diff_coefficient = []
for i in range(group_num):
  diff_coefficient.append((graph_array_std[i]/graph_array_mean[i])*100)

for i in range(len(sorted_df)-int(group_num)):
  diff_coefficient.append(0)
sorted_df['Difference Coefficient'] = diff_coefficient

#------------------visualization---------------------------------
## FOR VISUALIZATION GRAPH
x = []
for i in range(group_num):
  x.append("Group "+ str(i+1))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, graph_array_mean, color='blue', yerr=graph_array_std)
plt.xlabel("Group Number")
plt.ylabel("Tumor Volume")
plt.title("Average Tumor Volume")

plt.xticks(x_pos, x)

plt.show()
#------------------saving the file---------------------------------
print('\n')
filename = input('Enter output filename (ie. output.csv): ')
print('Filename: '+ filename)
sorted_df.to_csv(filename)
files.download(filename)



In [ ]:
#@title TRP 4 Groups

import pandas as pd
import numpy as np
import io
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

input('Hit ENTER to run program')

print("Welcome to the Tumor Randomization Program")
print('\n')
#print("Please enter the file name below like this example --> wlabdata.xlsx")

from google.colab import files
uploaded = files.upload()

file_path = 'randomization_file.xlsx'
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
file_path

#pd.set_option('max_rows', 10)
#pd.set_option('display.max_columns', None)

df = pd.read_excel(io.BytesIO(uploaded[str(file_path)]))
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
print(df)
#------------------volume column---------------------------------
print('\n')
#vol_col = df["volume"]
volumes = df['volume']
volumes
#------------------ID column---------------------------------
id_col = df['Combined ID']
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
array = []
for i in df['volume']:
    array.append(i)
#------------------sorting array and formatting---------------------------------
array.sort()
array
new_array = []
for i in range(len(array)):
    new_array.append(array[len(array)-i-1])
    new_array.append(array[i])
    i+=1
new_array
val1 = len(array)
val2 = len(new_array)
new_array = np.delete(new_array, [range(val1,val2)])

new_df = pd.DataFrame(new_array)
new_df
#------------------grouping---------------------------------
#grouping
#group_num = input("Please enter number of groups: ")
group_num = int(4)
groups = []
new_group = [1,2,3,4,4,3,2,1]
group_intermediate = []
group_intermediate.append(new_group*len(df['volume']))

groups = []
for i in range(len(df['volume'])):
  groups.append(group_intermediate[0][i])

new_df["Groups"] = groups
print(new_df)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#sorting
sorted_df = new_df.sort_values("Groups")

df.index = df['volume']
id_df = df.drop(['volume'], axis = 'columns')
id_dict = id_df.to_dict()
print('--------ID DICT--------')
print(id_dict)
id_array = []
for i in sorted_df[0]:
  id_array.append(id_dict['Combined ID'][i])

sorted_df['Mouse ID'] = id_array
print(sorted_df)

stat_df = sorted_df.groupby('Groups')
#------------------statistics---------------------------------
mean = stat_df.mean()
mean = mean.to_numpy()
mean = mean.tolist()
median = stat_df.median()
median = median.to_numpy()
median = median.tolist()
std_dev = stat_df.std()
std_dev = std_dev.to_numpy()
std_dev = std_dev.tolist()

for i in range(len(sorted_df.index)-len(mean)):
  mean.append(0)
  median.append(0)
  std_dev.append(0)

sorted_df['Mean'] = mean
sorted_df['Median'] = median
sorted_df['Std_Dev'] = std_dev

#------------------ANOVA---------------------------------
##Stuff for the anova
anova_df = sorted_df.groupby('Groups', 0)
med = []
for i in anova_df:
  med.append(i)

anova_stat = []
for i in range(group_num):
  thing = med[i][1][0].to_numpy()
  anova_stat.append(thing)

anova_stat

print('Follow instructions below for ANOVA testing')
print('Within the code below, change the f_oneway variable to calculate the an accurate statistic.')
print('The number of inputs depends on how many groups you have.')
print('\n')
arr = np.zeros(len(groups))
###-----------------------------------------MAKE SURE THE LINE BELOW IS RIGHT FOR ANOVA TESTING---------------------------------------------
anova_fvalue, anova_pvalue = f_oneway(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3])
print("ANOVA p-value: " + str(anova_pvalue))
arr[0] = anova_pvalue
#print(arr)
sorted_df['ANOVA'] = arr
#------------------Kruskal-Wallis---------------------------------
##COPY AND PASTE THE VALUES FROM THE F_ONEWAY LINE INSIDE THE KRUSKAL LINE PLEASE
arr1 = np.zeros(len(groups))
kru_fvalue, kru_pvalue = stats.kruskal(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3])
print("Kruskal p-value: " + str(kru_pvalue))
arr1[0] = kru_pvalue
sorted_df['Kruskal Wallis'] = arr1

#------------------percentage difference---------------------
avg = sorted_df[0].mean()
avg

graph_array_mean = []
for i in range(group_num):
  graph_array_mean.append(mean[i][0])
graph_array_std = []
for i in range(group_num):
  graph_array_std.append(std_dev[i][0])

diff_array = []
for i in graph_array_mean:
  diff = avg - i
  val = abs(diff/avg)*100
  diff_array.append(val)

for i in range(len(sorted_df)-int(group_num)):
  diff_array.append(0)

sorted_df['Difference Percentages'] = diff_array
#------------------difference coefficient---------------------------------
#difference coefficient
diff_coefficient = []
for i in range(group_num):
  diff_coefficient.append((graph_array_std[i]/graph_array_mean[i])*100)

for i in range(len(sorted_df)-int(group_num)):
  diff_coefficient.append(0)
sorted_df['Difference Coefficient'] = diff_coefficient

#------------------visualization---------------------------------
## FOR VISUALIZATION GRAPH
x = []
for i in range(group_num):
  x.append("Group "+ str(i+1))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, graph_array_mean, color='blue', yerr=graph_array_std)
plt.xlabel("Group Number")
plt.ylabel("Tumor Volume")
plt.title("Average Tumor Volume")

plt.xticks(x_pos, x)

plt.show()
#------------------saving the file---------------------------------
print('\n')
filename = input('Enter output filename (ie. output.csv): ')
print('Filename: '+ filename)
sorted_df.to_csv(filename)
files.download(filename)



In [ ]:
#@title TRP 5 Groups

import pandas as pd
import numpy as np
import io
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

input('Hit ENTER to run program')

print("Welcome to the Tumor Randomization Program")
print('\n')
#print("Please enter the file name below like this example --> wlabdata.xlsx")

from google.colab import files
uploaded = files.upload()

file_path = 'randomization_file.xlsx'
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
file_path

#pd.set_option('max_rows', 10)
#pd.set_option('display.max_columns', None)

df = pd.read_excel(io.BytesIO(uploaded[str(file_path)]))
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
print(df)
#------------------volume column---------------------------------
print('\n')
#vol_col = df["volume"]
volumes = df['volume']
volumes
#------------------ID column---------------------------------
id_col = df['Combined ID']
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
array = []
for i in df['volume']:
    array.append(i)
#------------------sorting array and formatting---------------------------------
array.sort()
array
new_array = []
for i in range(len(array)):
    new_array.append(array[len(array)-i-1])
    new_array.append(array[i])
    i+=1
new_array
val1 = len(array)
val2 = len(new_array)
new_array = np.delete(new_array, [range(val1,val2)])

new_df = pd.DataFrame(new_array)
new_df
#------------------grouping---------------------------------
#grouping
#group_num = input("Please enter number of groups: ")
group_num = int(5)

groups =[]
group_vals = []
for i in range(group_num):
  group_vals.append(i+1)
group_vals
group_inter = []
group_inter.append(group_vals*len(df['volume']))
for i in range(len(df['volume'])):
  groups.append(group_inter[0][i])

new_df["Groups"] = groups
print(new_df)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#sorting
sorted_df = new_df.sort_values("Groups")

df.index = df['volume']
id_df = df.drop(['volume'], axis = 'columns')
id_dict = id_df.to_dict()
print('--------ID DICT--------')
print(id_dict)
id_array = []
for i in sorted_df[0]:
  id_array.append(id_dict['Combined ID'][i])

sorted_df['Mouse ID'] = id_array
print(sorted_df)

stat_df = sorted_df.groupby('Groups')
#------------------statistics---------------------------------
mean = stat_df.mean()
mean = mean.to_numpy()
mean = mean.tolist()
median = stat_df.median()
median = median.to_numpy()
median = median.tolist()
std_dev = stat_df.std()
std_dev = std_dev.to_numpy()
std_dev = std_dev.tolist()

for i in range(len(sorted_df.index)-len(mean)):
  mean.append(0)
  median.append(0)
  std_dev.append(0)


sorted_df['Mean'] = mean
sorted_df['Median'] = median
sorted_df['Std_Dev'] = std_dev

#------------------ANOVA---------------------------------
##Stuff for the anova
anova_df = sorted_df.groupby('Groups', 0)
med = []
for i in anova_df:
  med.append(i)

anova_stat = []
for i in range(group_num):
  thing = med[i][1][0].to_numpy()
  anova_stat.append(thing)

anova_stat

print('Follow instructions below for ANOVA testing')
print('Within the code below, change the f_oneway variable to calculate the an accurate statistic. Input should look like: f_oneway(anova_stat[0], anova_stat[1], etc...')
print('The number of inputs depends on how many groups you have.')
print('\n')
arr = np.zeros(len(groups))
###-----------------------------------------MAKE SURE THE LINE BELOW IS RIGHT FOR ANOVA TESTING---------------------------------------------
anova_fvalue, anova_pvalue = f_oneway(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4])
print("ANOVA p-value: " + str(anova_pvalue))
arr[0] = anova_pvalue
#print(arr)
sorted_df['ANOVA'] = arr
#------------------Kruskal-Wallis---------------------------------
##COPY AND PASTE THE VALUES FROM THE F_ONEWAY LINE INSIDE THE KRUSKAL LINE PLEASE
arr1 = np.zeros(len(groups))
kru_fvalue, kru_pvalue = stats.kruskal(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4])
print("Kruskal p-value: " + str(kru_pvalue))
arr1[0] = kru_pvalue
sorted_df['Kruskal Wallis'] = arr1

#------------------percentage difference---------------------
avg = sorted_df[0].mean()
avg

graph_array_mean = []
for i in range(group_num):
  graph_array_mean.append(mean[i][0])
graph_array_std = []
for i in range(group_num):
  graph_array_std.append(std_dev[i][0])

diff_array = []
for i in graph_array_mean:
  diff = avg - i
  val = abs(diff/avg)*100
  diff_array.append(val)

for i in range(len(sorted_df)-int(group_num)):
  diff_array.append(0)

sorted_df['Difference Percentages'] = diff_array
#------------------difference coefficient---------------------------------
#difference coefficient
diff_coefficient = []
for i in range(group_num):
  diff_coefficient.append((graph_array_std[i]/graph_array_mean[i])*100)

for i in range(len(sorted_df)-int(group_num)):
  diff_coefficient.append(0)
sorted_df['Difference Coefficient'] = diff_coefficient

#------------------visualization---------------------------------
## FOR VISUALIZATION GRAPH
x = []
for i in range(group_num):
  x.append("Group "+ str(i+1))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, graph_array_mean, color='blue', yerr=graph_array_std)
plt.xlabel("Group Number")
plt.ylabel("Tumor Volume")
plt.title("Average Tumor Volume")

plt.xticks(x_pos, x)

plt.show()
#------------------saving the file---------------------------------
print('\n')
filename = input('Enter output filename (ie. output.csv): ')
print('Filename: '+ filename)
sorted_df.to_csv(filename)
files.download(filename)



In [ ]:
#@title TRP 6 Groups

import pandas as pd
import numpy as np
import io
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

input('Hit ENTER to run program')

print("Welcome to the Tumor Randomization Program")
print('\n')
#print("Please enter the file name below like this example --> wlabdata.xlsx")

from google.colab import files
uploaded = files.upload()

file_path = 'randomization_file.xlsx'
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
file_path

#pd.set_option('max_rows', 10)
#pd.set_option('display.max_columns', None)

df = pd.read_excel(io.BytesIO(uploaded[str(file_path)]))
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
print(df)
#------------------volume column---------------------------------
print('\n')
#vol_col = df["volume"]
volumes = df['volume']
volumes
#------------------ID column---------------------------------
id_col = df['Combined ID']
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
array = []
for i in df['volume']:
    array.append(i)
#------------------sorting array and formatting---------------------------------
array.sort()
array
new_array = []
for i in range(len(array)):
    new_array.append(array[len(array)-i-1])
    new_array.append(array[i])
    i+=1
new_array
val1 = len(array)
val2 = len(new_array)
new_array = np.delete(new_array, [range(val1,val2)])

new_df = pd.DataFrame(new_array)
new_df
#------------------grouping---------------------------------
#grouping
#group_num = input("Please enter number of groups: ")
group_num = int(6)
groups = []
new_group = [1,2,3,4,5,6,6,5,4,3,2,1]
group_intermediate = []
group_intermediate.append(new_group*len(df['volume']))

groups = []
for i in range(len(df['volume'])):
  groups.append(group_intermediate[0][i])

new_df["Groups"] = groups
print(new_df)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#sorting
sorted_df = new_df.sort_values("Groups")

df.index = df['volume']
id_df = df.drop(['volume'], axis = 'columns')
id_dict = id_df.to_dict()
print('--------ID DICT--------')
print(id_dict)
id_array = []
for i in sorted_df[0]:
  id_array.append(id_dict['Combined ID'][i])

sorted_df['Mouse ID'] = id_array
print(sorted_df)

stat_df = sorted_df.groupby('Groups')
#------------------statistics---------------------------------
mean = stat_df.mean()
mean = mean.to_numpy()
mean = mean.tolist()
median = stat_df.median()
median = median.to_numpy()
median = median.tolist()
std_dev = stat_df.std()
std_dev = std_dev.to_numpy()
std_dev = std_dev.tolist()

for i in range(len(sorted_df.index)-len(mean)):
  mean.append(0)
  median.append(0)
  std_dev.append(0)

sorted_df['Mean'] = mean
sorted_df['Median'] = median
sorted_df['Std_Dev'] = std_dev

#------------------ANOVA---------------------------------
##Stuff for the anova
anova_df = sorted_df.groupby('Groups', 0)
med = []
for i in anova_df:
  med.append(i)

anova_stat = []
for i in range(group_num):
  thing = med[i][1][0].to_numpy()
  anova_stat.append(thing)

anova_stat

print('Follow instructions below for ANOVA testing')
print('Within the code below, change the f_oneway variable to calculate the an accurate statistic. Input should look like: f_oneway(anova_stat[0], anova_stat[1], etc...')
print('The number of inputs depends on how many groups you have.')
print('\n')
arr = np.zeros(len(groups))
###-----------------------------------------MAKE SURE THE LINE BELOW IS RIGHT FOR ANOVA TESTING---------------------------------------------
anova_fvalue, anova_pvalue = f_oneway(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4], anova_stat[5])
print("ANOVA p-value: " + str(anova_pvalue))
arr[0] = anova_pvalue
#print(arr)
sorted_df['ANOVA'] = arr
#------------------Kruskal-Wallis---------------------------------
##COPY AND PASTE THE VALUES FROM THE F_ONEWAY LINE INSIDE THE KRUSKAL LINE PLEASE
arr1 = np.zeros(len(groups))
kru_fvalue, kru_pvalue = stats.kruskal(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4], anova_stat[5])
print("Kruskal p-value: " + str(kru_pvalue))
arr1[0] = kru_pvalue
sorted_df['Kruskal Wallis'] = arr1

#------------------percentage difference---------------------
avg = sorted_df[0].mean()
avg

graph_array_mean = []
for i in range(group_num):
  graph_array_mean.append(mean[i][0])
graph_array_std = []
for i in range(group_num):
  graph_array_std.append(std_dev[i][0])

diff_array = []
for i in graph_array_mean:
  diff = avg - i
  val = abs(diff/avg)*100
  diff_array.append(val)

for i in range(len(sorted_df)-int(group_num)):
  diff_array.append(0)

sorted_df['Difference Percentages'] = diff_array
#------------------difference coefficient---------------------------------
#difference coefficient
diff_coefficient = []
for i in range(group_num):
  diff_coefficient.append((graph_array_std[i]/graph_array_mean[i])*100)

for i in range(len(sorted_df)-int(group_num)):
  diff_coefficient.append(0)
sorted_df['Difference Coefficient'] = diff_coefficient

#------------------visualization---------------------------------
## FOR VISUALIZATION GRAPH
x = []
for i in range(group_num):
  x.append("Group "+ str(i+1))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, graph_array_mean, color='blue', yerr=graph_array_std)
plt.xlabel("Group Number")
plt.ylabel("Tumor Volume")
plt.title("Average Tumor Volume")

plt.xticks(x_pos, x)

plt.show()
#------------------saving the file---------------------------------
print('\n')
filename = input('Enter output filename (ie. output.csv): ')
print('Filename: '+ filename)
sorted_df.to_csv(filename)
files.download(filename)



In [ ]:
#@title TRP 7 Groups

import pandas as pd
import numpy as np
import io
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

input('Hit ENTER to run program')

print("Welcome to the Tumor Randomization Program")
print('\n')
#print("Please enter the file name below like this example --> wlabdata.xlsx")

from google.colab import files
uploaded = files.upload()

file_path = 'randomization_file.xlsx'
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
file_path

#pd.set_option('max_rows', 10)
#pd.set_option('display.max_columns', None)

df = pd.read_excel(io.BytesIO(uploaded[str(file_path)]))
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
print(df)
#------------------volume column---------------------------------
print('\n')
#vol_col = df["volume"]
volumes = df['volume']
volumes
#------------------ID column---------------------------------
id_col = df['Combined ID']
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
array = []
for i in df['volume']:
    array.append(i)
#------------------sorting array and formatting---------------------------------
array.sort()
array
new_array = []
for i in range(len(array)):
    new_array.append(array[len(array)-i-1])
    new_array.append(array[i])
    i+=1
new_array
val1 = len(array)
val2 = len(new_array)
new_array = np.delete(new_array, [range(val1,val2)])

new_df = pd.DataFrame(new_array)
new_df
#------------------grouping---------------------------------
#grouping
#group_num = input("Please enter number of groups: ")
group_num = int(7)

groups =[]
group_vals = []
for i in range(group_num):
  group_vals.append(i+1)
group_vals
group_inter = []
group_inter.append(group_vals*len(df['volume']))
for i in range(len(df['volume'])):
  groups.append(group_inter[0][i])

new_df["Groups"] = groups
print(new_df)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#sorting
sorted_df = new_df.sort_values("Groups")

df.index = df['volume']
id_df = df.drop(['volume'], axis = 'columns')
id_dict = id_df.to_dict()
print('--------ID DICT--------')
print(id_dict)
id_array = []
for i in sorted_df[0]:
  id_array.append(id_dict['Combined ID'][i])

sorted_df['Mouse ID'] = id_array
print(sorted_df)

stat_df = sorted_df.groupby('Groups')
#------------------statistics---------------------------------
mean = stat_df.mean()
mean = mean.to_numpy()
mean = mean.tolist()
median = stat_df.median()
median = median.to_numpy()
median = median.tolist()
std_dev = stat_df.std()
std_dev = std_dev.to_numpy()
std_dev = std_dev.tolist()

for i in range(len(sorted_df.index)-len(mean)):
  mean.append(0)
  median.append(0)
  std_dev.append(0)

sorted_df['Mean'] = mean
sorted_df['Median'] = median
sorted_df['Std_Dev'] = std_dev

#------------------ANOVA---------------------------------
##Stuff for the anova
anova_df = sorted_df.groupby('Groups', 0)
med = []
for i in anova_df:
  med.append(i)

anova_stat = []
for i in range(group_num):
  thing = med[i][1][0].to_numpy()
  anova_stat.append(thing)

anova_stat

print('Follow instructions below for ANOVA testing')
print('Within the code below, change the f_oneway variable to calculate the an accurate statistic. Input should look like: f_oneway(anova_stat[0], anova_stat[1], etc...')
print('The number of inputs depends on how many groups you have.')
print('\n')
arr = np.zeros(len(groups))
###-----------------------------------------MAKE SURE THE LINE BELOW IS RIGHT FOR ANOVA TESTING---------------------------------------------
anova_fvalue, anova_pvalue = f_oneway(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4], anova_stat[5], anova_stat[6])
print("ANOVA p-value: " + str(anova_pvalue))
arr[0] = anova_pvalue
#print(arr)
sorted_df['ANOVA'] = arr
#------------------Kruskal-Wallis---------------------------------
##COPY AND PASTE THE VALUES FROM THE F_ONEWAY LINE INSIDE THE KRUSKAL LINE PLEASE
arr1 = np.zeros(len(groups))
kru_fvalue, kru_pvalue = stats.kruskal(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4], anova_stat[5], anova_stat[6])
print("Kruskal p-value: " + str(kru_pvalue))
arr1[0] = kru_pvalue
sorted_df['Kruskal Wallis'] = arr1

#------------------percentage difference---------------------
avg = sorted_df[0].mean()
avg

graph_array_mean = []
for i in range(group_num):
  graph_array_mean.append(mean[i][0])
graph_array_std = []
for i in range(group_num):
  graph_array_std.append(std_dev[i][0])

diff_array = []
for i in graph_array_mean:
  diff = avg - i
  val = abs(diff/avg)*100
  diff_array.append(val)

for i in range(len(sorted_df)-int(group_num)):
  diff_array.append(0)

sorted_df['Difference Percentages'] = diff_array
#------------------difference coefficient---------------------------------
#difference coefficient
diff_coefficient = []
for i in range(group_num):
  diff_coefficient.append((graph_array_std[i]/graph_array_mean[i])*100)

for i in range(len(sorted_df)-int(group_num)):
  diff_coefficient.append(0)
sorted_df['Difference Coefficient'] = diff_coefficient

#------------------visualization---------------------------------
## FOR VISUALIZATION GRAPH
x = []
for i in range(group_num):
  x.append("Group "+ str(i+1))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, graph_array_mean, color='blue', yerr=graph_array_std)
plt.xlabel("Group Number")
plt.ylabel("Tumor Volume")
plt.title("Average Tumor Volume")

plt.xticks(x_pos, x)

plt.show()
#------------------saving the file---------------------------------
print('\n')
filename = input('Enter output filename (ie. output.csv): ')
print('Filename: '+ filename)
sorted_df.to_csv(filename)
files.download(filename)



In [ ]:
#@title TRP 8 Groups

import pandas as pd
import numpy as np
import io
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

input('Hit ENTER to run program')

print("Welcome to the Tumor Randomization Program")
print('\n')
#print("Please enter the file name below like this example --> wlabdata.xlsx")

from google.colab import files
uploaded = files.upload()

file_path = 'randomization_file.xlsx'
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
file_path

#pd.set_option('max_rows', 10)
#pd.set_option('display.max_columns', None)

df = pd.read_excel(io.BytesIO(uploaded[str(file_path)]))
#'C://Users//nkdge//Desktop//wlabdata.xlsx'
print(df)
#------------------volume column---------------------------------
print('\n')
#vol_col = df["volume"]
volumes = df['volume']
volumes
#------------------ID column---------------------------------
id_col = df['Combined ID']
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
array = []
for i in df['volume']:
    array.append(i)
#------------------sorting array and formatting---------------------------------
array.sort()
array
new_array = []
for i in range(len(array)):
    new_array.append(array[len(array)-i-1])
    new_array.append(array[i])
    i+=1
new_array
val1 = len(array)
val2 = len(new_array)
new_array = np.delete(new_array, [range(val1,val2)])

new_df = pd.DataFrame(new_array)
new_df
#------------------grouping---------------------------------
#grouping
#group_num = input("Please enter number of groups: ")
group_num = int(8)
groups = []
new_group = [1,2,3,4,5,6,7,8,8,7,6,5,4,3,2,1]
group_intermediate = []
group_intermediate.append(new_group*len(df['volume']))

groups = []
for i in range(len(df['volume'])):
  groups.append(group_intermediate[0][i])

new_df["Groups"] = groups
print(new_df)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#sorting
sorted_df = new_df.sort_values("Groups")

df.index = df['volume']
id_df = df.drop(['volume'], axis = 'columns')
id_dict = id_df.to_dict()
print('--------ID DICT--------')
print(id_dict)
id_array = []
for i in sorted_df[0]:
  id_array.append(id_dict['Combined ID'][i])

sorted_df['Mouse ID'] = id_array
print(sorted_df)

stat_df = sorted_df.groupby('Groups')
#------------------statistics---------------------------------
mean = stat_df.mean()
mean = mean.to_numpy()
mean = mean.tolist()
median = stat_df.median()
median = median.to_numpy()
median = median.tolist()
std_dev = stat_df.std()
std_dev = std_dev.to_numpy()
std_dev = std_dev.tolist()

for i in range(len(sorted_df.index)-len(mean)):
  mean.append(0)
  median.append(0)
  std_dev.append(0)

sorted_df['Mean'] = mean
sorted_df['Median'] = median
sorted_df['Std_Dev'] = std_dev

#------------------ANOVA---------------------------------
##Stuff for the anova
anova_df = sorted_df.groupby('Groups', 0)
med = []
for i in anova_df:
  med.append(i)

anova_stat = []
for i in range(group_num):
  thing = med[i][1][0].to_numpy()
  anova_stat.append(thing)

anova_stat

print('Follow instructions below for ANOVA testing')
print('Within the code below, change the f_oneway variable to calculate the an accurate statistic. Input should look like: f_oneway(anova_stat[0], anova_stat[1], etc...')
print('The number of inputs depends on how many groups you have.')
print('\n')
arr = np.zeros(len(groups))
###-----------------------------------------MAKE SURE THE LINE BELOW IS RIGHT FOR ANOVA TESTING---------------------------------------------
anova_fvalue, anova_pvalue = f_oneway(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4], anova_stat[5])
print("ANOVA p-value: " + str(anova_pvalue))
arr[0] = anova_pvalue
#print(arr)
sorted_df['ANOVA'] = arr
#------------------Kruskal-Wallis---------------------------------
##COPY AND PASTE THE VALUES FROM THE F_ONEWAY LINE INSIDE THE KRUSKAL LINE PLEASE
arr1 = np.zeros(len(groups))
kru_fvalue, kru_pvalue = stats.kruskal(anova_stat[0],anova_stat[1], anova_stat[2], anova_stat[3], anova_stat[4], anova_stat[5])
print("Kruskal p-value: " + str(kru_pvalue))
arr1[0] = kru_pvalue
sorted_df['Kruskal Wallis'] = arr1

#------------------percentage difference---------------------
avg = sorted_df[0].mean()
avg

graph_array_mean = []
for i in range(group_num):
  graph_array_mean.append(mean[i][0])
graph_array_std = []
for i in range(group_num):
  graph_array_std.append(std_dev[i][0])

diff_array = []
for i in graph_array_mean:
  diff = avg - i
  val = abs(diff/avg)*100
  diff_array.append(val)

for i in range(len(sorted_df)-int(group_num)):
  diff_array.append(0)

sorted_df['Difference Percentages'] = diff_array
#------------------difference coefficient---------------------------------
#difference coefficient
diff_coefficient = []
for i in range(group_num):
  diff_coefficient.append((graph_array_std[i]/graph_array_mean[i])*100)

for i in range(len(sorted_df)-int(group_num)):
  diff_coefficient.append(0)
sorted_df['Difference Coefficient'] = diff_coefficient

#------------------visualization---------------------------------
## FOR VISUALIZATION GRAPH
x = []
for i in range(group_num):
  x.append("Group "+ str(i+1))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, graph_array_mean, color='blue', yerr=graph_array_std)
plt.xlabel("Group Number")
plt.ylabel("Tumor Volume")
plt.title("Average Tumor Volume")

plt.xticks(x_pos, x)

plt.show()
#------------------saving the file---------------------------------
print('\n')
filename = input('Enter output filename (ie. output.csv): ')
print('Filename: '+ filename)
sorted_df.to_csv(filename)
files.download(filename)



